In [ ]:
"""learning_finesse.ipynb
James Gardner 2020

This was used while going through Sections 1 and 2 of the tutorials on gwoptics
http://www.gwoptics.org/learn/index.html

Tasks are separated by an empty cell
"""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse                  
from pykat.commands import *              


pykat.init_pykat_plotting(dpi=90)

In [ ]:
# base model on which to make changes
basekat = finesse.kat()
basekat.verbose = False

In [ ]:
basecode = """
l laser 1 0 n0   # Laser (Power = 1 W, wavelength offset = 0)
s s1 1 n0 nc1    # Space (Length = 1 m)

# the cavity
m m1 0.7 0.3 0 nc1 nc2  # Mirror  (R = 0.7, T = 0.3, phi = 0)
s sL 4000 nc2 nc3       # Space (Length = 4 km)
m m2 0.8 0.2 0 nc3 nc4  # Mirror  (R = 0.8, T = 0.2, phi = 0)

# detectors
pd circ nc2
pd tran nc4
pd refl nc1
"""

basekat.parse(basecode)

In [ ]:
print(basekat.components,basekat.detectors,basekat.commands,sep='\n')

In [ ]:
# laser offset frequency
kat1 = deepcopy(basekat)
kat1.laser.f = 2e4
kat1code = """
xaxis sL L lin 0 10k 2000
# yaxis abs
"""

kat1.parse(kat1code)
out1 = kat1.run()
out1.plot(ylim=(0,None))

In [ ]:
# changing T1, fixing T1 + T2
kat1 = deepcopy(basekat)

code = """
## Simulation instructions ##
xaxis m1 T lin 0 0.6 200   # Varying transmittance of input mirror m1.
yaxis abs                  # Plotting the amplitude of the detector measurements. 

## Functions computing reflectance and transmittance ##
func R1 = 1-$x1             # Reflectance of m1 (lossless)
func T2 = 0.6 - $x1         # Transmittance of m2 (T1 + T2 = 0.6)
func R2 = 1-$T2             # Reflectance of m2

## Setting values ## 
put m1 R $R1                # Setting reflectance of m1 to R1
put m2 T $T2                # Setting transmittance of m2 to T2
put m2 R $R2                # Setting reflectance of m2 to R2
"""

kat1.parseKatCode(code) # Parsing the FINESSE-code
out1 = kat1.run() 

fig1a = out1.plot(detectors=['refl','circ','tran'], 
                  xlabel='T1', 
                  ylabel='Power [W]', 
                  title = 'Power vs. T1',
                  ylim = (0,None))

# Reflectances and transmittances
fig1b = out1.plot(detectors=['R1','R2','T2'], 
                  xlabel='T1', 
                  ylabel='R, T', title='R1, R2, and T2 vs. T1')

In [ ]:
kat2 = deepcopy(basekat)

code = """
ad a_refl 0 nc1
ad a_tran 0 nc4
ad a_circ 0 nc2

xaxis m2 phi lin -180 180 2000
yaxis abs:deg
"""

kat2.parse(code)

In [ ]:
T_tot = 0.3

# Over-coupled, T1 > T2
T1 = 0.25
T2 = T_tot-T1
R1 = 1-T1
R2 = 1-T2
kat2.m1.T = T1
kat2.m1.R = R1
kat2.m2.T = T2
kat2.m2.R = R2
out2a = kat2.run()

# Under-coupled, T1 < T2
T1 = 0.05
T2 = T_tot-T1
R1 = 1-T1
R2 = 1-T2
kat2.m1.T = T1
kat2.m1.R = R1
kat2.m2.T = T2
kat2.m2.R = R2
out2b = kat2.run()

# Impedance matched, T1 = T2
T1 = 0.15
T2 = T_tot-T1
R1 = 1-T1
R2 = 1-T2
kat2.m1.T = T1
kat2.m1.R = R1
kat2.m2.T = T2
kat2.m2.R = R2
out2c = kat2.run() 

fig2a = out2a.plot(detectors=['a_circ','a_refl','a_tran'],
                   title = 'Over-coupled cavity, $T_1>T_2$',
                   styles={'a_circ':'b-','a_refl':'r-','a_tran':'k--'})

fig2b = out2b.plot(detectors=['a_circ','a_refl','a_tran'],
                  ylim = (0,None),
                  styles = {'a_circ':'b-','a_refl':'r-','a_tran':'k--'},
                  title = 'undercoupled')
fig2c = out2c.plot(detectors=['a_circ','a_refl','a_tran'],
                  ylim = (0,None),
                  styles = {'a_circ':'b-','a_refl':'r-','a_tran':'k--'},
                  title = 'impedance matched')

In [ ]:
kat3 = deepcopy(basekat)
code = """
xaxis laser f lin -5k 90k 1000
yaxis abs
"""
kat3.parse(code)

In [ ]:
# 1st run, original cavity length of 4 km.
out3a = kat3.run()

# 2nd run, cavity length of 6 km.
kat3.sL.L = 6000
out3b = kat3.run()

# 3rd run, cavity length of 8 km.
kat3.sL.L = 8000
out3c = kat3.run()

# Run 1
fig3a = out3a.plot(xlabel='Frequency [Hz]', 
                   ylabel='Power [W]', 
                   title='Run 1: Cavity length = 4 km',
                   loc=1)
# Run 2
fig3b = out3b.plot(xlabel='Frequency [Hz]', 
                   ylabel='Power [W]', 
                   title='Run 2: Cavity length = 6 km', 
                   loc=1)
# Run 3
fig3c = out3c.plot(xlabel='Frequency [Hz]', 
                   ylabel='Power [W]', 
                   title='Run 3: Cavity length = 8 km', 
                   loc = 1)

In [ ]:
# ignoring optimisation task because it looks pretty tedious
# future James: turns out we ended up doing optimisation, see below

In [ ]:
ifobasekat = finesse.kat()

code = """
## Parameters ##
const Pin 1M         # Laser power (1 MW)
const LX 4000        # Length of X arm (4000 m)
const LY 4000        # Length of Y arm (4000 m)

## Laser & Beam splitter ##
l laser $Pin 0 n0                         # Laser
s s1 1 n0 nbsc                            # Space from laser to beam splitter
bs BS 0.5 0.5 0 45 nbsc nbsY nbsX nbsd    # Beam splitter (R = T = 0.5, phi = 0 deg, 
                                          # AoI = 45 deg).
## X arm ##
s LX $LX nbsX nmX1         # Space, BS to mX (4000 m)
m mX 1 0 0 nmX1 nmX2       # Mirror mX (R = 1, T = 0, phi = 0 deg)

## Y arm ##
s LY $LY nbsY nmY1         # Space, BS to mY (4000 m)
m mY 1 0 0 nmY1 nmY2       # Mirror mY (R = 1, T = 0, phi = 0 deg)

## Output port ##
s sout 1 nbsd nout         # Space, BS to output
pd pout nout

"""
# """
# # constants
# const Pin 1M
# const LX 4k
# const LY 4k

# # components
# l laser Pin 0 n1
# s s1 1 n1 nbs1
# bs BS 0.5 0.5 0 45 nbs1 nbs2 nbs3 nbs4
# s sx LX nbs2 nm1
# m m1 0.9 0.1 0 nm1 nm1o
# s sy LY nbs3 nm2
# m m2 0.9 0.1 0 nm2 nm2o

# # detectors
# pd darkport nbs4
# pd brightport nbs1
# """

ifobasekat.parse(code)

In [ ]:
print(ifobasekat.components,ifobasekat.detectors,ifobasekat.constants,sep='\n')

In [ ]:
# differential tuning
kat1 = deepcopy(ifobasekat)
code = """
xaxis mX phi lin -180 180 2000
put* mY phi $mx1
yaxis abs
"""
kat1.parse(code)

In [ ]:
out1 = kat1.run()

fig1 = out1.plot(xlabel="DARM / 2 [deg]", 
                 ylabel="Power [W]", 
                 title="Output power vs. Arm length difference")

In [ ]:
kat2 = deepcopy(kat1)
# changing mX is pointless given xaxis, instead change mY and be sure to use put*
kat2.mY.phi = 90

out2 = kat2.run()
out2.plot()

In [ ]:
import scipy.optimize as opt
help(opt.minimize_scalar)

In [ ]:
# why not larger than 10mW?
kat3 = deepcopy(kat2)
code = """
noxaxis
"""
kat3.parse(code)
out3 = kat3.run()

# out3.y gives abs at darkport

In [ ]:
def darkport(phi):
    kat = deepcopy(kat2)
    kat.noxaxis = True
    kat.mY.phi = phi
    out = kat.run()
    return out.y[0][0]

In [ ]:
dp10m = lambda x : darkport(x) - 10e-3

In [ ]:
val10m = opt.bisect(dp10m, 0, 90)

In [ ]:
darkport(val10m)

In [ ]:
kat3c = deepcopy(kat3)
kat3c.mY.phi = val10m
out3c = kat3c.run()
print('Pout = {0} mW'.format(1000.0*out3c['pout']))

In [ ]:
ifobasekat.components

In [ ]:
# differential length
kat4 = deepcopy(ifobasekat)
kat4.laser.f = 1e5
code = """
xaxis* LX L lin -1k 1k 200  # * avoids -ve lengths
func yshift = 4000 + $mx1
put* LY L $yshift
noplot yshift
yaxis abs:deg
"""
kat4.parse(code)
out4 = kat4.run()
out4.plot()

In [ ]:
# modulation
basekat = finesse.kat()
code= """
## The optical system ##
l laser 1 0 n0                # 1 W laser with 0 offset wavelength
s s1 1 n0 n1                  # Space from laser to mirror (1 m)
mod eom 1k 0.3 5 pm 0 n1 n2   # Modulator (f=1 kHz, m=0.3, 5 sideband pairs, 
                              # phase modulation, 0 degree modulation phase)
s s2 1000 n2 n3               # Space from modulator to output (1 m)
"""
basekat.parse(code)

In [ ]:
kat1 = basekat.deepcopy()

code = """
## Detectors ##
ad car 0 n3                   # Ampitude dector, carrier (0 freq offset)
ad up1 1k n3                  # Ampitude dector, first upper sideband (1 kHz offset)
ad up2 2k n3                  # Ampitude dector, second upper sideband (2 kHz offset)
ad up3 3k n3                  # Ampitude dector, third upper sideband (3 kHz offset)
ad up4 4k n3                  # Ampitude dector, fourth upper sideband (4 kHz offset)
ad up5 5k n3                  # Ampitude dector, fourth upper sideband (5 kHz offset)

## Instructions ##
xaxis eom midx lin 0 6 200    # Varying the modulation index between 0 and 1
yaxis abs                   # Showing amplitude detected by amplitude detectors
"""

# Parsing the code and running the simulation
kat1.parseCommands(code)
out1 = kat1.run()

In [ ]:
fig1 = out1.plot(title = 'Amplitude of freq. components vs. modulation index', 
                 ylabel='Amplitude $[\sqrt{W}]$',  
                 xlabel='Modulation index')

In [ ]:
# Copying our core model
kat2 = basekat.deepcopy()

# Adding detectors and instructions
code = """
## Detectors ##

# Amplitude detectors
ad car 0 n3                   # For the carrier (0 freq offset)
ad up1 1k n3                  # For 1st upper sideband (1 kHz freq offset)
ad low1 -1k n3                # For 1st lower sideband (-1 kHz freq offset)
ad up2 2k n3                  # For 2nd upper sideband (2 kHz freq offset)
ad low2 -2k n3                # For 2nd lower sideband (-2 kHz freq offset)
ad up3 3k n3                  # For 3rd upper sideband (3 kHz freq offset)
ad low3 -3k n3                # For 3rd lower sideband (-3 kHz freq offset)
ad up4 4k n3                  # For 4th upper sideband (4 kHz freq offset)
ad low4 -4k n3                # For 4th lower sideband (-4 kHz freq offset)
ad up5 5k n3                  # For 5th upper sideband (5 kHz freq offset)
ad low5 -5k n3                # For 5th lower sideband (-5 kHz freq offset)

# Photo diode
pd pow n3                     # Detecting DC-power

## Instructions ##
xaxis eom midx lin 0 6 200    # Varying modulation index between 0 and 1
yaxis abs                     # Absolute value of the detector outputs
"""

# Parsing the code and running the simulation
kat2.parseKatCode(code)
out2 = kat2.run()

In [ ]:
# was missing lower sidebands, plus amplitude isn't conserved, energy is |E|^2
# Computing power
# ---------------
P1 = (out2.y[:,0:3]**2).sum(axis=1)    # With one sideband pair
P2 = (out2.y[:,0:5]**2).sum(axis=1)    # With two sideband pairs
P3 = (out2.y[:,0:7]**2).sum(axis=1)    # With three sideband pairs
P4 = (out2.y[:,0:9]**2).sum(axis=1)    # With four sideband pairs
P5 = (out2.y[:,0:11]**2).sum(axis=1)   # With five sideband pairs

# Plotting
# ---------
fig2 = plt.figure(figsize=(8,4))
ax = plt.subplot(1,1,1)
ax.plot(out2.x, P1, label='1 sb pairs')
ax.plot(out2.x, P2, label='2 sb pairs')
ax.plot(out2.x, P3, label='3 sb pairs')
ax.plot(out2.x, P4, label='4 sb pairs')
ax.plot(out2.x, P5, label='5 sb pairs')
ax.plot(out2.x, out2['pow'],'k--', label='DC Power')
# Labels and legend
ax.set_title('Power conservation of EOM')
ax.set_ylabel('Power [W]')
ax.set_xlabel('Modulation index')
ax.legend(loc=3)
plt.show(fig2)

In [ ]:
basekat = finesse.kat()          # Initialising Finesse
basekat.verbose = False          # Tells Finesse to talk less

# Finesse code
code = """
## The optical system ##
l laser 1 0 n0                # 1 W laser, 0 wavelength offset (using default 1064nm)
s s1 1 n0 n1                  # Space from laser to mirror (1 m)
mod eom 1k 0.3 1 am 0 n1 n2   # Modulator (f=1 kHz, m=0.3, 1 sideband pair, 
                              # amplitude modulation, 0 degree mod. phase)
s s2 1000 n2 n3               # Space from modulator to output (1 m)
"""

# Parsing the Finesse code
basekat.parseKatCode(code)

In [ ]:
kat1 = deepcopy(basekat)

code = """
## Detectors ##
ad car 0 n3                   # Ampitude dector, carrier (0 freq offset)
ad up1 1k n3                   # Ampitude dector, upper sideband (1 kHz offset)
ad low1 -1k n3                   # Ampitude dector, lower sideband (-1 kHz offset)

pd pow n3                     # Photo diode, detecting DC-power

## Simulation Instructions ##
xaxis eom midx lin 0 1 200   # Varying the modulation index between 0 and 1.
yaxis abs                    # Returns magnitudes of detector outputs
"""

# Parsing the code and running the simulation
kat1.parseCommands(code)
out1 = kat1.run()

In [ ]:
# Only plotting the 
fig1 = out1.plot(detectors=['car','low1','up1'],
                 title = 'Amplitude of freq. components vs. modulation index', 
                 ylabel='Amplitude $[\sqrt{W}]$',  
                 xlabel='Modulation index')

In [ ]:
total_energy = (out1.y[:,0:3]**2).sum(axis=1)
plt.plot(out1.x, total_energy)

In [ ]:
basekat = finesse.kat()    # Initialising FINESSE
basekat.verbose = False    # Keep quiet

# FINESSE code
code = '''
const freq 10k             # creating a constant for the frequency offset

l laser1 1 0 n1            # laser with P=1W at the default frequency
s s1 1n n1 n2              # space of 1nm length

l laser2 0.1 $freq n3      # a second laser with f=10kHz frequency offset
s s2 1n n3 n4              # another space of 1nm length

bs bs1 0.5 0.5 0 0 n2 n5 dump n4   # 50:50 beam splitter
s s3 1n n5 n6                      # another space of 1nm length
'''
basekat.parse(code)

In [ ]:
# Copying basekat
kat = deepcopy(basekat)

# FINESSE code
code = '''
const freq 10k             
## Detectors ##
ad a1 0 n6                        # amplitude detector, default frequency
ad a2 $freq n6                    # amplitude detector, 10 kHz offset
pd dc n6                          # photo didoe, measuring DC-power
pd1 ac $freq n6                   # photo diode + demodulation at 10kHz + 
                                  # + low pass filter

## Simulation instructions ##
xaxis laser2 phi lin 0 180 200     # Sweeping phase of beam from laser2
yaxis abs:deg                      # Returns complex representation of detector outputs
'''

# Running the simulation
kat.parse(code)
out = kat.run()

In [ ]:
# Setting 
styles={'a1':'b-','a2':'k-','dc':'g--','ac':'r--'}

fig = out.plot(styles = styles, 
               xlabel='Phase of laser2 [deg]',
               loc=1)

In [ ]:
# Copying basekat
kat = deepcopy(basekat)

# FINESSE code
code = '''
const freq 10k             
## Detectors ##
ad a1 0 n6                        # amplitude detector, default frequency
ad a2 $freq n6                    # amplitude detector, 10 kHz offset
pd dc n6                          # photo didoe, measuring DC-power
pd1 ac $freq n6                   # photo diode + demodulation at 10kHz + 
                                  # + low pass filter

## Simulation instructions ##
xaxis laser2 P log 1p 1M 2000     # Sweeping phase of beam from laser2
yaxis log abs                     # Returns complex representation of detector outputs

set pow dc abs                    # making output of dc available as variable 
                                  # with name pow.
set ac1 ac abs                    # making output of ac avaiblable as variable 
                                  # with name ac1.
func E2 = $ac1/sqrt($pow+1E-21)   # Recreating amplitude of laser2. 1E-21 is used 
                                  # in denominator to avoid division by zero.
'''

# Running the simulation
kat.parse(code)
out = kat.run()

In [ ]:
# Setting plotting style for the detectors

# remember that laser 1 is at 1W of power, which gives the crossing point below
style = {'a1':'b-','a2':'y--','dc':'g--','ac':'r--','E2':'k-'}
fig2 = out.plot(styles = style, 
                 xlabel='Power of laser2 [W]', 
                 ylabel='Power [W],   Amplitude [$\sqrt{\mathrm{W}}$]',
                 loc=4)

In [ ]:
basekat = finesse.kat()          # Initialising Finesse
basekat.verbose = False          # Tells Finesse to talk less

# Parsing the code
code = """
## The optical system ##
l laser 1 0 n0                               # Laser (Power = 1W, Freq offset = 0 Hz)
s s1 1 n0 nBS1                               # Space, laser to BS (L = 1 m)
bs BS1 0.5 0.5 0 45 nBS1 dump nBS3 nBS4      # Beam splitter (R=T=0.5, phi=0 deg, 
                                             # AoI=45 deg)
s s2 1 nBS3 nM1                              # Space, BS to Mirror (L = 1 m)
m M1 1 0 0 nM1 nM2                           # Mirror (R=1, T=0, phi=0 deg)
s s3 1 nBS4 nOut                             # Space, BS to detectors (L = 1m)

## Shaking the mirror longitudinally ##
fsig sig1 M1 100 0 1                         # Signal (f=100 Hz, phase=0 deg, 
                                             # amplitude=1)
"""
# Parsing FINESSE-code
basekat.parse(code)

In [ ]:
kat1 = basekat.deepcopy()
code = """
## Detectors ##

# Amplitude detectors
ad car 0 nOut             # Carrier (Offset = 0 Hz)
ad up1 100 nOut           # 1st upper sideband (Offset = 100 Hz)
ad low1 -100 nOut         # 1st lower sideband (Offset = -100 Hz)
ad up2 200 nOut           # 2nd upper sideband (Offset = 200 Hz)
#ad low2 -200 nOut           


## Simulation instructions ##
xaxis sig1 amp lin 0.01 2 200   # Sweeping amplitude of sig1 between 0.01 and 2. 
yaxis abs
"""

# Parsing FINESSE code and running
kat1.parse(code)
out1 = kat1.run()

In [ ]:
## Plotting ##
# -------------
fig1 = out1.plot(styles={'car': '-k','up1':'--r', 'low1': '-b',  'up2':'g-'},
                 xlabel='Mirror motion amplitude [rad]',
                 ylabel='Reflected Amplitude [$\sqrt{\mathrm{W}}$]',
                 title='Sideband amplitude vs. mirror motion, using fsig', 
                 ylim=(-0.05,1))

In [ ]:
kat2 = deepcopy(basekat)
kat2.M1.R = 0.9
kat2.M1.T = 0.1
code = """
## Detectors ##

# Amplitude detectors
ad car 0 nM2             # Carrier (Offset = 0 Hz)
ad up1 100 nM2           # 1st upper sideband (Offset = 100 Hz)
ad low1 -100 nM2         # 1st lower sideband (Offset = -100 Hz)
ad up2 200 nM2           # 2nd upper sideband (Offset = 200 Hz)
#ad low2 -200 nM2           


## Simulation instructions ##
xaxis sig1 amp lin 0.01 2 200   # Sweeping amplitude of sig1 between 0.01 and 2. 
yaxis abs
"""

# Parsing FINESSE code and running
kat2.parse(code)
out2 = kat2.run()
out2.plot()

In [ ]:
basekat = finesse.kat()          # Initialising Finesse
basekat.verbose = False          # Tells Finesse to talk less

basecode = """
## The optical system ##

l laser 1 0 n0                # Laser (P = 1 W, Freq. offset = 0 Hz)
s s1 1 n0 n1                  # Space, laser to EOM (L = 1 m)
mod eom 1k 0.3 1 am 30 n1 n2  # Modulator (f = 1 kHz, m=0.3, 1 sideband pair, 
                              # type = amplitude modulation, phase = 30 deg.)
s s2 1n n2 n3                 # Space, EOM to output (L = 1 nm)

"""
# Parsing the FINESSE-code
basekat.parse(basecode)

In [ ]:
kat1 = basekat.deepcopy()

code = """
## Detectors ##
pd1 inphase 1k 0 n3     # Photodiode + mixer + low pass filter 
                        # (demod. f = 1kHz, demod phase = 0 deg.)
pd1 inquad 1k 90 n3     # Photodiode + mixer + low pass filter 
                        # (demod. f = 1kHz, demod. phase = 90 deg.)
pd1 optimal 1k 30 n3    # Photodiode + mixer + low pass filter 
                        # (demod. f = 1kHz, demod. phase = 30 deg.)

## Simulation Instructions ##
xaxis eom midx lin 0 1 200   # Varying EOM modulation between 0 and 1
yaxis abs                # Returns absolute values of detector outputs
"""

# Parsing the code and running the simulation
kat1.parseCommands(code)
out1 = kat1.run()

fig1a = out1.plot(xlabel = 'Modulation index',
                  ylabel = 'Power [W]',
                  styles = {'optimal':'g--'})

In [ ]:
# Reconstructing full amplitude and phase information
tot = out1['inphase'] + 1j*out1['inquad']

## Plotting ##
fig1b = plt.figure(figsize = (8,4))
ax = plt.subplot(1,1,1)
ax.plot(out1.x, out1['inphase'], 'b-', label='In-phase (0 deg)')
ax.plot(out1.x, out1['inquad'], 'g--', label='In-quadrature (90 deg)')
ax.plot(out1.x, out1['optimal'], 'ro', label='Optimal')
ax.plot(out1.x, np.abs(tot), 'k--', label='Reconstructed')

ax.set_xlabel('Modulation index $m$')
ax.set_ylabel('Power [W]')
ax.set_title('Demodulation')
ax.legend(loc=0, fontsize=10)

plt.show()

In [ ]:
kat2 = basekat.deepcopy()

code = """
## Detectors ##
pd1 inphase 1k 0 n3     # Photodiode + mixer + low pass filter 
                        # (demod. f = 1kHz, demod phase = 0 deg.)
pd1 inquad 1k 90 n3     # Photodiode + mixer + low pass filter 
                        # (demod. f = 1kHz, demod. phase = 90 deg.)
pd dcpower n3
ad amp 1k n3

## Simulation Instructions ##
xaxis eom midx lin 0 1 200   # Varying EOM modulation between 0 and 1
yaxis abs:deg                # Returns absolute values of detector outputs
"""

# Parsing the code and running the simulation
kat2.parse(code)
out2 = kat2.run()

In [ ]:
# Reconstructing full amplitude and phase information

out2.plot(styles={'amp':'b--','dcpower':'r.'})

In [ ]:
tot = out2['inphase'] + 1j*out2['inquad']

# Calculating the amplitude of the carrier
a0 = np.sqrt( ( out2['dcpower'] + np.sqrt(out2['dcpower']**2 - 2*np.abs(tot)**2) )/2 )

# Calculating complex amplitude of the upper sideband
a_plus = tot/(2*a0)

## Plotting ##
fig2, (ax, ax2) = plt.subplots(2, 1, figsize = (10,10), sharex=True)

# ax.plot(out2.x, out2['inphase'], 'b-', label='In-phase (0 deg)')
# ax.plot(out2.x, out2['inquad'], 'g--', label='In-quadrature (90 deg)')
ax.plot(out2.x, np.abs(a_plus), 'ko', label='abs Reconstructed')
ax.plot(out2.x, np.abs(out2['amp']), 'w--', label='abs amp')

ax2.plot(out2.x, np.angle(a_plus), 'ko', label='phi Reconstructed')
ax2.plot(out2.x, np.angle(out2['amp']), 'w--', label='angle amp')

ax2.set_xlabel('Modulation index $m$')
ax.set_ylabel('Power [W]')
ax.set_title('Demodulation')
ax.legend(loc=0, fontsize=10)

plt.show()

In [ ]:
basekat = finesse.kat()          # Initialising Finesse
basekat.verbose = False          # Tells Finesse to talk less

basecode = """
## The optical system ##

l laser 100 0 n0               # Laser (P = 100 W, Freq. offset = 0 Hz)
s s1 1 n0 n1                   # Space, laser to EOM (L = 1 m)
mod eom1 1M 0.5 1 am 20 n1 n2  # Modulator (f = 1 MHz, m=0.5, 1 sideband pair, 
                               # type = amplitude modulation, phase = 20 deg.)
s s2 1 n2 n3                   # Space, eom1 to eom2 (L = 1 m)
mod eom2 1k 0.3 1 am 60 n3 n4  # Modulator (f = 1 kHz, m=0.3, 1 sideband pair, 
                               # type = amplitude modulation, phase = 60 deg.)
s s3 1 n4 n5                   # Space, eom2 to output (L = 1 m)

"""
# Parsing the FINESSE-code
basekat.parse(basecode)

In [ ]:
kat1a = basekat.deepcopy()

code = """
## Detectors ##
pd1 inphase 1M 0 n5    # Photodiode + mixer + low pass filter 
                       # (f1 = 1 MHz, phi1 = 0 deg)
pd1 inquad 1M 90 n5    # Photodiode + mixer + low pass filter 
                       # (f1 = 1 MHz, phi1 = 90 deg)

## Simulation Instructions ##
noxaxis                # No xaxis. Computing just one data point.
yaxis abs              # Returns absolute values of detector outputs
"""

# Parsing the code and running the simulation
kat1a.parseCommands(code)
out1a = kat1a.run()

In [ ]:
# Reconstructing full complex demodulation signal
S1 = out1a['inphase'] + 1j*out1a['inquad']
# Calculating the phase
phase1 = np.angle(S1)*180.0/np.pi
print('Optimal demodulation phase at 1 MHz: {:f} deg'.format(phase1))

In [ ]:
kat1b = basekat.deepcopy()

code = """
## Detectors ##
pd2 inphase 1M {0} 1k 0 n5     # Photodiode + mixer + low pass filter 
                               # (f1 = 1 MHz, phi1 = 20 deg, 
                               #  f2 = 1 kHz, phi2 = 0 deg)
pd2 inquad 1M {0} 1k 90 n5     # Photodiode + mixer + low pass filter 
                               # (f1 = 1 MHz, phi1 = 20 deg, 
                               #  f2 = 1 kHz, phi2 = 90 deg)
pd2 optimal 1M {0} 1k max n5   # Photodiode + mixer + low pass filter 
                               # (f1 = 1 MHz, phi1 = 20 deg, 
                               #  f2 = 1 kHz, phi2 = max)

## Simulation Instructions ##
xaxis eom2 midx lin 0 1 200    # Varying EOM modulation between 0 and 1
yaxis abs                      # Returns absolute values of detector outputs
""".format(phase1) #<--- super cool, python wrapper onto Finesse code directly!

# Parsing the code and running the simulation
kat1b.parseCommands(code)
out1b = kat1b.run()
# Plotting the detector outputs
fig1b = out1b.plot(xlabel='Modulation index of eom2', 
                   ylabel='Power [W]',
                   styles={'optimal':'--'})

In [ ]:
# Reconstructing
S2 = out1b['inphase'] + 1j*out1b['inquad']

## Plotting ##

# Plotting the the absolute value
fig1c = plt.figure(figsize = (8,3))
ax = plt.subplot(1,1,1)
ax.plot(out1b.x, np.abs(out1b['optimal']), 
        'r-', label='Optimal')
ax.plot(out1b.x, np.abs(S2), 
        'k--', label='Reconstructed')
ax.set_ylabel('Power [W]')
ax.set_xlabel('Modulation index of eom2')
ax.set_title('')
ax.legend(loc=0, fontsize=10)
'''
# Plotting the phase (Converting from radians to degrees)
ax2 = plt.subplot(2,1,2)
ax2.plot(out1b.x, np.angle(out1b['optimal'])*180/np.pi, 'r-', label='Optimal')
ax2.plot(out1b.x, np.angle(S2)*180/np.pi, 'k--', label='Reconstructed')
ax2.set_xlabel('Modulation index of eom2')
ax2.set_ylabel('Phase [deg]')
ax2.set_ylim((0,70))
ax2.legend(loc=0, fontsize=10)
'''
plt.show()

# Calculating and printing the phase of S2 in degrees
phase2 = np.angle(S2[1])*180/np.pi
print('Phase of S2: {:.4f} deg'.format(phase2))

In [ ]:
# now that we know the optimal value, we can just use that as phi_x instead of requiring quadrature breakdown
kat2 = basekat.deepcopy()

code = """
## Detectors ##
pd2 inphase 1M {0} 1k 0 n5     # Photodiode + mixer + low pass filter 
                               # (f1 = 1 MHz, phi1 = 20 deg, 
                               #  f2 = 1 kHz, phi2 = 0 deg)
pd2 inquad 1M {0} 1k 90 n5     # Photodiode + mixer + low pass filter 
                               # (f1 = 1 MHz, phi1 = 20 deg, 
                               #  f2 = 1 kHz, phi2 = 90 deg)

pd2 amp 1M {0} 1k n5

xaxis eom2 midx lin 0 1 200 
yaxis abs:deg   
""".format(phase1) #<--- super cool, python wrapper onto Finesse code directly!

# Parsing the code and running the simulation
kat2.parse(code)
out2 = kat2.run()
# Plotting the detector outputs
out2.plot()

In [ ]:
# Reconstructing
S2 = out2['inphase'] + 1j*out2['inquad']
cheat = out2['amp']

visA = 'k-'
visB = 'r--'
lw = 4

# Plotting the the absolute value
fig2, (ax, ax2) = plt.subplots(2, 1, figsize = (10,10))

ax.plot(out2.x, np.abs(cheat), 
        visA, label='cheat', linewidth=lw)
ax.plot(out2.x, np.abs(S2), 
        visB, label='Reconstructed', linewidth=lw)
ax.set_ylabel('Power [W]')
ax.set_xlabel('Modulation index of eom2')
ax.set_title('')
ax.legend(loc=0, fontsize=10)

# Plotting the phase (Converting from radians to degrees)
ax2.plot(out1b.x, np.angle(cheat)*180/np.pi, visA, label='cheat', linewidth=lw)
ax2.plot(out1b.x, np.angle(S2)*180/np.pi, visB, label='Reconstructed', linewidth=lw)

ax2.set_xlabel('Modulation index of eom2')
ax2.set_ylabel('Phase [deg]')
ax2.set_ylim((0,70))
ax2.legend(loc=0, fontsize=10)

plt.show()

# Calculating and printing the phase of S2 in degrees
phase2 = np.angle(S2[1])*180/np.pi
phase2_cheat = np.angle(cheat[1])*180/np.pi
print('recon. phase of S2: {0:.20f} deg\n cheat phase of S2: {1:.20f} deg'.format(phase2,phase2_cheat))

In [ ]:
"""old code"""

# import pykat as pk
# %matplotlib inline

# kat = pk.finesse.kat()
# kat.load('cavity1.kat')
# print(kat.components,kat.detectors,kat.commands)
# print(kat.components['i1'])
# kat.m2.R = 0.9
# kat.m2.T = 0.1

# code = """
# l laser 1 0 n1
# s space 1 n1 n2
# pd pout n2
# xaxis laser P lin 1 10 100
# """

# kat.parse(code)

# out = kat.run()
# out.plot()
